In [1]:
import os
os.environ["OPENAI_API_KEY"] = "sk---"

In [23]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters.character import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain_openai import ChatOpenAI
from langchain_core.prompts.prompt import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough


In [3]:
loader = PyPDFLoader("loveandfriendship.pdf")
pages = []
async for page in loader.alazy_load():
    pages.append(page)

In [4]:
pages

[Document(metadata={'source': 'loveandfriendship.pdf', 'page': 0}, page_content='Love and Friendship\nand\nOther Writings\nJane Austen\nONEWORLD\nCLASSICS'),
 Document(metadata={'source': 'loveandfriendship.pdf', 'page': 1}, page_content='oneworld  classics  ltd\nLondon House\n243-253 Lower Mortlake Road\nRichmond\nSurrey TW9 2LL\nUnited Kingdom\nwww .oneworldclassics.com\nThis edition first published by Oneworld Classics Limited in 2009\nNotes and Extra material © Oneworld Classics, 2009\nPrinted in Great Britain by CPI Anthony Rowe\nisbn: 978-1-84749-092-6\nAll the material in this volume is reprinted with permission or presumed to be \nin the public domain. Every effort has been made to ascertain and acknowledge \nthe copyright status, but should there have been any unwitting oversight on our \npart, we would be happy to rectify the error in subsequent printings.\nAll rights reserved. No part of this publication may be reproduced, stored in or \nintroduced into a retrieval system, o

In [7]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=300,
    chunk_overlap=5,
    separators = ["\n\n","\n","."]
)

docs = text_splitter.split_documents(pages)

In [8]:
docs

[Document(metadata={'source': 'loveandfriendship.pdf', 'page': 0}, page_content='Love and Friendship\nand\nOther Writings\nJane Austen\nONEWORLD\nCLASSICS'),
 Document(metadata={'source': 'loveandfriendship.pdf', 'page': 1}, page_content='oneworld  classics  ltd\nLondon House\n243-253 Lower Mortlake Road\nRichmond\nSurrey TW9 2LL\nUnited Kingdom\nwww .oneworldclassics.com\nThis edition first published by Oneworld Classics Limited in 2009\nNotes and Extra material © Oneworld Classics, 2009\nPrinted in Great Britain by CPI Anthony Rowe'),
 Document(metadata={'source': 'loveandfriendship.pdf', 'page': 1}, page_content='isbn: 978-1-84749-092-6\nAll the material in this volume is reprinted with permission or presumed to be \nin the public domain. Every effort has been made to ascertain and acknowledge \nthe copyright status, but should there have been any unwitting oversight on our'),
 Document(metadata={'source': 'loveandfriendship.pdf', 'page': 1}, page_content='part, we would be happy to

In [10]:

vectorstore = Chroma.from_documents(documents=docs, embedding=OpenAIEmbeddings())

In [11]:
# retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 6})

retriever = vectorstore.as_retriever(search_type="mmr", search_kwargs={"k": 5})

In [13]:

llm = ChatOpenAI(temperature=0)
retriever_from_llm = MultiQueryRetriever.from_llm(
    retriever=retriever, llm=llm
)

In [14]:
query ="What happened one evening in December while Laura father, mother, and Laura were conversing around the fireside?"

In [15]:

retrieved_docs = retriever_from_llm.invoke(query)


In [28]:
retrieved_docs

[Document(metadata={'page': 9, 'source': 'loveandfriendship.pdf'}, page_content='Laura\nletter  the fifth\nLaura to Marianne\nOne evening in December as my father, my mother and myself were \narranged in social converse round our fireside, we were of a sudden \ngreatly astonished by hearing a violent knocking on the outward door \nof our rustic cottage.'),
 Document(metadata={'page': 10, 'source': 'loveandfriendship.pdf'}, page_content='who had lost their way , were very cold, and begged leave to warm \nthemselves by our fire.\n“Won’t you admit them?” said my father.\n“Y ou have no objection, my dear?” said my father.\n“None in the world,” replied my mother.\nMary , without waiting for any further commands, immediately left'),
 Document(metadata={'page': 16, 'source': 'loveandfriendship.pdf'}, page_content='affected the delicate sensibility of Edward, Sophia, your Laura, and \nof Augustus himself. T o complete such unparalleled barbarity , we \nwere informed that an execution in the ho

In [19]:
template = """You are good at summarising the conversation OR a story. 
So, please use the below context and summarize without missing any pointers
context: {context}
---------
query: {query}
"""
prompt = PromptTemplate.from_template(template)

In [20]:
model = ChatOpenAI(temperature=0.8)

In [35]:
final_context = ""
for doc in retrieved_docs:
    final_context = final_context+doc.page_content

In [40]:
chain = (
    RunnablePassthrough()  # Initial passthrough without direct input
    | (lambda x: {"context": final_context, "query": x})  # Lambda to create dict input
    | prompt
    | model
    | StrOutputParser()
)

In [43]:
res = chain.invoke({"query":query})


In [45]:
print(res)

Laura, her father, and mother were engaged in social conversation around the fireside one evening in December when they were startled by a violent knocking on the door of their cottage. Two individuals who had lost their way and were very cold requested to warm themselves by the fire, to which Laura's father and mother agreed. However, the situation took a distressing turn when it was revealed that an execution was to take place in the house. Overwhelmed by the news, Laura and her family could only sigh and faint on the sofa. Subsequently, they found themselves alone in the room, missing both their family members and valuable banknotes.


Laura, her father, and mother were engaged in social conversation around the fireside one evening in December when they were startled by a violent knocking on the door of their cottage. Two individuals who had lost their way and were very cold requested to warm themselves by the fire, to which Laura's father and mother agreed. However, the situation took a distressing turn when it was revealed that an execution was to take place in the house. Overwhelmed by the news, Laura and her family could only sigh and faint on the sofa. Subsequently, they found themselves alone in the room, missing both their family members and valuable banknotes.
